# Example 2.3

In [1]:
import numpy as np
from z3 import *

# Truth table vectors
tt_x          = np.array([0,0,1,1])
tt_y          = np.array([0,1,0,1])
tt_x_xor_y    = np.array([0,1,1,0])
tt_x_or_not_y = np.array([1,0,1,1])
tt_neg_1      = np.array([1,1,1,1])

# Convert into matrix from column vectors
F = np.column_stack((tt_x, tt_y, tt_x_xor_y, tt_x_or_not_y, tt_neg_1))

# Define incognites for solution
x1, x2, x3, x4, x5 = Ints('x1 x2 x3 x4 x5')
V = [x1, x2, x3, x4, x5]

# Initialize the Z3 solver engine
solver = Solver()

# Add the conditions for the system of equations
print ("System of equations:\n---")
for row in range(F.shape[0]):
    eq = F[row][0]*V[0]
    for col in range(1, F.shape[1]):
        eq += F[row][col]*V[col]
    
    print(f"{eq} = 0")
    solver.add(eq == 0)
    
# We don't want the trivial solution
solver.add(
    Not(
        And(
            x1 == 0, x2 == 0, x3 == 0, x4 == 0, x5 == 0
        )
    )
)

# Check and extract the model (i.e. a particular solution)
solver.check()
m = solver.model()

# Sort and print the output of the model nicely
ordered = (sorted (m.decls(), key = lambda x: str(x)))
print("\nCandidate solution:\n---\nV =", ordered, '=', [m[i] for i in ordered])

# Define x and y as BitVectors of 8 bits (you can change the bitsize)
x, y = BitVecs('x y', 8)

# We prove that the derived MBA expression is non-trivially equal to zero
print("\nCheck that MBA expression is non-trivially equal to zero:\n---")
prove(-x + y + (x^y) + 2*(x | (~y)) + 2 == 0)

System of equations:
---
0*x1 + 0*x2 + 0*x3 + 1*x4 + 1*x5 = 0
0*x1 + 1*x2 + 1*x3 + 0*x4 + 1*x5 = 0
1*x1 + 0*x2 + 1*x3 + 1*x4 + 1*x5 = 0
1*x1 + 1*x2 + 0*x3 + 1*x4 + 1*x5 = 0

Candidate solution:
---
V = [x1, x2, x3, x4, x5] = [-1, 1, 1, 2, -2]

Check that MBA expression is non-trivially equal to zero:
---
proved
